Loading all the libraries. Keeping them up here and will update as I add them.

In [7]:
import csv
import numpy as np
import math as ma
import pandas as pd

Loading the data set. This will be where I load all of them.

In [6]:
vg_chartz = pd.read_csv(r'vgchartz-7_7_2020.csv')

Making a copy of the vg_chartz, so I can play around with it as needed.

In [8]:
# creating copy to do most of the exploratory work with
vg_copy = vg_chartz.copy()

In [10]:
# Just checking out how many unique publishers there are.
len(pd.unique(vg_chartz['publisher']))

3261

Based on the number of publishers, I will need to find a way to link them to the company that owns them.

In [17]:
# removing series and visual novels, might use merge series later as a data point, so making its own df
vg_copy = vg_copy[vg_copy.console != 'Series']
vg_copy = vg_copy[vg_copy.genre != 'Visual Novel']
vg_series = vg_chartz[vg_chartz.console == 'Series']

In [19]:
vg_series.head

<bound method NDFrame.head of        Unnamed: 0                                            img  \
0               3  /games/boxart/full_3351915AmericaFrontccc.jpg   
1               4  /games/boxart/full_5741036AmericaFrontccc.jpg   
2               5  /games/boxart/full_6662824AmericaFrontccc.png   
4               7  /games/boxart/full_6800951AmericaFrontccc.jpg   
16             19  /games/boxart/full_8886048AmericaFrontccc.jpg   
...           ...                                            ...   
54132       54978  /games/boxart/full_7404545AmericaFrontccc.jpg   
54133       54979  /games/boxart/full_7398718AmericaFrontccc.png   
54139       54985  /games/boxart/full_5242617AmericaFrontccc.jpg   
54142       54988  /games/boxart/full_9421541AmericaFrontccc.jpg   
54189       55035  /games/boxart/full_9984096AmericaFrontccc.jpg   

                               title console     genre  \
0                           Warriors  Series    Action   
1                         God of War 

280 series, which should be workable. I probably wont work with all of them, since some of them probably are just the same game released for different systems (eg. Tetris, Frogger, The Binding of Isaac, etc.) For these games, the total shipped could be an interesting stat to consider.

In [14]:
# checking out the header
vg_copy.head

<bound method NDFrame.head of        Unnamed: 0                                             img  \
3               6   /games/boxart/full_6510540AmericaFrontccc.jpg   
5               8   /games/boxart/full_5563178AmericaFrontccc.jpg   
6               9                    /games/boxart/3887656ccc.jpg   
7              10   /games/boxart/full_2804791AmericaFrontccc.jpg   
8              11                     /games/boxart/827563ccc.jpg   
...           ...                                             ...   
57523       58420   /games/boxart/full_6832495AmericaFrontccc.jpg   
57524       58421  /games/boxart/full_8405567AmericaFrontccc.jpeg   
57525       58422   /games/boxart/full_1661793AmericaFrontccc.png   
57526       58423   /games/boxart/full_5465265AmericaFrontccc.jpg   
57527       58424   /games/boxart/full_6380526AmericaFrontccc.jpg   

                                                 title console     genre  \
3                                   Grand Theft Auto V     PS3   

With 57428 rows and 18 columns, there is definitely something to look at. Will be focusing on critic score and total sales (or shipped) for continuous variables.

Categorical will be publisher, console, and genre. Might add some series analysis later.

Will look at stocks for a few of the publishers, probably focusing on the ones that only produce game software. (No Nintendo, Sony, or Microsoft.) Will probably look at Activision, EA, Ubisoft, Take-Two, and Square-Enix. The problem with that will probably be combining some of the companies that have multiple publishers (eg. Take-Two)

In [15]:
# counting publishers
vg_copy['publisher'].nunique()

3240

Count of unique publishers = 3240: might need to delve into that more later, removing publishers that focus on one or two titles. It could be an interesting data point later, seeing if a publisher correlates with another measurement variable.

In [56]:
# grouping stuff

group = vg_copy.groupby(['genre'])
group.agg(['mean', 'std', 'count'])['critic_score']

,mean,std,count
genre,,,
Action,6.840234,1.575293,1024
Action-Adventure,7.665657,1.239170,99
Adventure,7.329188,1.455842,394
Board Game,5.300000,NaN,1
Education,7.500000,NaN,1
Fighting,7.388415,1.383228,328
MMO,7.810000,1.408269,10
Misc,6.885799,1.517741,338
Music,7.973077,0.961065,26


In [57]:
group.agg(['mean', 'std', 'count'])['total_sales']

,mean,std,count
genre,,,
Action,0.409260,1.048028,2880
Action-Adventure,0.556816,1.202643,267
Adventure,0.177714,0.396895,1920
Board Game,0.110000,0.164621,3
Education,0.090000,NaN,1
Fighting,0.400277,0.610933,903
MMO,0.370625,0.520452,32
Misc,0.289230,0.588833,2039
Music,0.352752,0.708438,149


In [100]:
# This was first try, using merging. Moved to using isin, below.
# group2 = vg_copy.groupby(['console'])
# test = group2.agg(['count'])['critic_score']
# blah = test > 100
# blah.reset_index()
# new_df = pd.merge(vg_copy, blah,  how='left', left_on=['console'], right_on=['console'])

In [122]:
# grouping it by column that interested in (console)
group2 = vg_copy.groupby(['console'])

# counting times that column (console here) (for each of its unique values) have a given value (critic score)
test = group2.agg(['count'])['critic_score']

# choosing the number of critic scores needed to be included
blah = test > 100

#only keeping the ones that have over that number of columns.
boo = blah[blah['count'] == True]

# now checking if those columns (consoles) are in the new 
vg_copy[vg_copy['console'].isin(boo.index)]

,Unnamed: 0,img,title,console,genre,publisher,developer,vg_score,critic_score,user_score,total_shipped,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
3,6,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,NaN,9.4,NaN,NaN,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
5,8,/games/boxart/full_5563178AmericaFrontccc.jpg,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,NaN,9.7,NaN,NaN,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
6,9,/games/boxart/3887656ccc.jpg,Grand Theft Auto: San Andreas,PS2,Action,Rockstar Games,Rockstar North,NaN,9.5,NaN,17.30,NaN,NaN,NaN,NaN,NaN,2004-10-26,NaN
7,10,/games/boxart/full_2804791AmericaFrontccc.jpg,Uncharted 4: A Thief's End,PS4,Action,Sony Interactive Entertainment,Naughty Dog,NaN,9.2,NaN,16.25,NaN,NaN,NaN,NaN,NaN,2016-05-10,2018-09-23
8,11,/games/boxart/827563ccc.jpg,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,NaN,9.6,NaN,NaN,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57521,58415,/games/boxart/full_3900223AmericaFrontccc.jpg,XCOM 2 Collection,NS,Strategy,2K Games,Firaxis Games,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-29,2020-03-31
57522,58416,/games/boxart/full_5852173AmericaFrontccc.jpg,XCOM: Chimera Squad,PC,Strategy,2K Games,Firaxis Games,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-24,2020-04-15
57524,58421,/games/boxart/full_8405567AmericaFrontccc.jpeg,Yu Yu Hakusho Tournament Tactics,GBA,Strategy,Atari,Sensory Sweep Studios,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004-11-18,2017-12-30
57525,58422,/games/boxart/full_1661793AmericaFrontccc.png,Yu-Gi-Oh! Legacy of the Duelist,PS4,Strategy,Konami,Other Ocean Interactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-31,2019-04-20


In [123]:
test

,count
console,
2600,0
3DO,0
3DS,220
5200,0
7800,0
...,...
XOne,58
XSX,0
ZXS,0


Created a function here that removes rows if not enough specific features.

In [142]:
def enough_stuff(dat_fr, uniq_col, count_col, cnt_val):
    """
    DOC STRING
    This function takes a data frame, a column of categoricals
    to count, and the number of items from that column that are 
    in rows needed to be kept. This is basically just removing 
    items that dont show up enough to study.
    
    Args:
    dat_fr: The data frame that will be looked at.
    uniq_col: The column of the data frame that the function will look at
    count_col: The column with the counting objects.
    cnt_val: The number of items needed to be included.
    
    
    Return:
    A new data frame with just the rows that have 
    those requested values in the column.
    A print out of how many rows were removed.
    """
    start_len = len(dat_fr)
    func_group = dat_fr.groupby([uniq_col])[count_col]
    func_TorF = func_group.agg(['count']) > cnt_val
    func_JustT = func_TorF[func_TorF['count'] == True]
    final_df = dat_fr[dat_fr[uniq_col].isin(func_JustT.index)]
    print(f'{start_len - len(final_df)} of {start_len} have been removed. You now have {len(final_df)}')
    return (final_df)
    

In [144]:
vg_copy = enough_stuff(vg_copy, 'console', 'critic_score', 100)

16032 of 57248 have been removed. You now have 41216


In [146]:
vg_copy = enough_stuff(vg_copy, 'genre', 'critic_score', 90)

1011 of 41216 have been removed. You now have 40205


In [51]:
vg_copy[vg_copy['title'].str.contains('Grand Theft Auto')]

,Unnamed: 0,img,title,console,genre,publisher,developer,vg_score,critic_score,user_score,total_shipped,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
3,6,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,NaN,9.4,NaN,NaN,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
5,8,/games/boxart/full_5563178AmericaFrontccc.jpg,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,NaN,9.7,NaN,NaN,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
6,9,/games/boxart/3887656ccc.jpg,Grand Theft Auto: San Andreas,PS2,Action,Rockstar Games,Rockstar North,NaN,9.5,NaN,17.3,NaN,NaN,NaN,NaN,NaN,2004-10-26,NaN
8,11,/games/boxart/827563ccc.jpg,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,NaN,9.6,NaN,NaN,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN
9,12,/games/boxart/full_9218923AmericaFrontccc.jpg,Grand Theft Auto V,X360,Action,Rockstar Games,Rockstar North,NaN,NaN,NaN,NaN,15.86,9.06,0.06,5.33,1.42,2013-09-17,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21156,21486,/games/boxart/default.jpg,Grand Theft Auto: San Andreas,And,Misc,Rockstar Games,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-01-01,NaN
21157,21487,/games/boxart/default.jpg,Grand Theft Auto: San Andreas,OSX,Misc,Rockstar Games,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-09-01,NaN
21158,21488,/games/boxart/default.jpg,Grand Theft Auto: San Andreas,WinP,Misc,Rockstar Games,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-01-01,NaN
21159,21489,/games/boxart/default.jpg,Grand Theft Auto: The Trilogy,PSV,Misc,Kadokawa Games,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,NaN


In [101]:

new_df = new_df[new_df['count'] == True]
new_df

,Unnamed: 0,img,title,console,genre,publisher,developer,vg_score,critic_score,user_score,total_shipped,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update,count
0,6,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,NaN,9.4,NaN,NaN,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN,True
1,8,/games/boxart/full_5563178AmericaFrontccc.jpg,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,NaN,9.7,NaN,NaN,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03,True
2,9,/games/boxart/3887656ccc.jpg,Grand Theft Auto: San Andreas,PS2,Action,Rockstar Games,Rockstar North,NaN,9.5,NaN,17.30,NaN,NaN,NaN,NaN,NaN,2004-10-26,NaN,True
3,10,/games/boxart/full_2804791AmericaFrontccc.jpg,Uncharted 4: A Thief's End,PS4,Action,Sony Interactive Entertainment,Naughty Dog,NaN,9.2,NaN,16.25,NaN,NaN,NaN,NaN,NaN,2016-05-10,2018-09-23,True
4,11,/games/boxart/827563ccc.jpg,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,NaN,9.6,NaN,NaN,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57241,58415,/games/boxart/full_3900223AmericaFrontccc.jpg,XCOM 2 Collection,NS,Strategy,2K Games,Firaxis Games,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-29,2020-03-31,True
57242,58416,/games/boxart/full_5852173AmericaFrontccc.jpg,XCOM: Chimera Squad,PC,Strategy,2K Games,Firaxis Games,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-24,2020-04-15,True
57244,58421,/games/boxart/full_8405567AmericaFrontccc.jpeg,Yu Yu Hakusho Tournament Tactics,GBA,Strategy,Atari,Sensory Sweep Studios,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004-11-18,2017-12-30,True
57245,58422,/games/boxart/full_1661793AmericaFrontccc.png,Yu-Gi-Oh! Legacy of the Duelist,PS4,Strategy,Konami,Other Ocean Interactive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-07-31,2019-04-20,True


In [68]:
counts = vg_copy['console'].nunique()
counts
# filtered = counts[counts > 100]
# vg3 = vg_copy[vg_copy['console'].isin(filtered.index)]
# group2 = vg3.groupby(['console'])
# group2.agg(['mean', 'std', 'count'])['critic_score']

77

In [72]:
vg_copy=vg_copy.loc[:,vg_copy.nunique()>100]
vg_copy

""


In [74]:
vg_copy = vg_copy.groupby('console')[].nunique()

KeyError: 'console'

In [99]:
# creating a groupby for publisher, so see stuff
vg_copy.groupby('publisher')

In [51]:
vg_copy[vg_copy['title'].str.contains('Grand Theft Auto')]

,Unnamed: 0,img,title,console,genre,publisher,developer,vg_score,critic_score,user_score,total_shipped,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
3,6,/games/boxart/full_6510540AmericaFrontccc.jpg,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,NaN,9.4,NaN,NaN,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
5,8,/games/boxart/full_5563178AmericaFrontccc.jpg,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,NaN,9.7,NaN,NaN,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
6,9,/games/boxart/3887656ccc.jpg,Grand Theft Auto: San Andreas,PS2,Action,Rockstar Games,Rockstar North,NaN,9.5,NaN,17.3,NaN,NaN,NaN,NaN,NaN,2004-10-26,NaN
8,11,/games/boxart/827563ccc.jpg,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,NaN,9.6,NaN,NaN,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN
9,12,/games/boxart/full_9218923AmericaFrontccc.jpg,Grand Theft Auto V,X360,Action,Rockstar Games,Rockstar North,NaN,NaN,NaN,NaN,15.86,9.06,0.06,5.33,1.42,2013-09-17,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21156,21486,/games/boxart/default.jpg,Grand Theft Auto: San Andreas,And,Misc,Rockstar Games,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-01-01,NaN
21157,21487,/games/boxart/default.jpg,Grand Theft Auto: San Andreas,OSX,Misc,Rockstar Games,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-09-01,NaN
21158,21488,/games/boxart/default.jpg,Grand Theft Auto: San Andreas,WinP,Misc,Rockstar Games,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-01-01,NaN
21159,21489,/games/boxart/default.jpg,Grand Theft Auto: The Trilogy,PSV,Misc,Kadokawa Games,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-01,NaN
